In [56]:
import os
import requests
from bs4 import BeautifulSoup
import matplotlib.pylab as plt
import utils
import warnings 
warnings.filterwarnings("ignore")
import pandas as pd 

# Q2a

In [36]:
def noaa_file_parser(filename):
    file_content = open(filename, "r").read().split("\n")
    # print(file_content)
    year = int(file_content[0].split(" ")[0])
    day = int(file_content[0].split(" ")[1])
    hour = int(file_content[0].split(" ")[2])
    
    l = []
    for item in file_content[1:]:
        tmp_l = item.split(" ")
        data_l = [i for i in tmp_l if i]
        if len(data_l) != 4:
            continue
        grid_number = int(data_l[0])
        wave_height = data_l[1]
        wave_direction = data_l[2]
        wave_period = data_l[3]
        l.append((filename, year, day, hour, grid_number, wave_height, wave_direction, wave_period))
    
    df = pd.DataFrame(l, columns=["filename", "year", "day", "hour", "grid_number", "wave_height", "wave_direction", "wave_period"])
    return df 

file_grid_data = noaa_file_parser(filename='o202101800.0.wav')
file_grid_data.head(10)


,filename,year,day,hour,grid_number,wave_height,wave_direction,wave_period
0,o202101800.0.wav,2021,18,1,1,0.416,78,2.4
1,o202101800.0.wav,2021,18,1,2,0.550,75,2.9
2,o202101800.0.wav,2021,18,1,3,0.639,76,3.1
3,o202101800.0.wav,2021,18,1,4,0.710,78,3.3
4,o202101800.0.wav,2021,18,1,5,0.256,130,2.0
5,o202101800.0.wav,2021,18,1,6,0.321,134,2.3
6,o202101800.0.wav,2021,18,1,7,0.359,132,2.4
7,o202101800.0.wav,2021,18,1,8,0.409,115,2.5
8,o202101800.0.wav,2021,18,1,9,0.527,100,2.8
9,o202101800.0.wav,2021,18,1,10,0.652,96,3.2


In [37]:
print("file_grid_data has {} rows".format(file_grid_data.shape[0]))


file_grid_data has 4476 rows


# Q2b

In [38]:
local_wav_files = [i for i in os.listdir("./") if "wav" in i]

In [39]:

tmp_df_list = []
for file in local_wav_files:
    tmp_df = noaa_file_parser(file)
    tmp_df_list.append(tmp_df)


In [40]:
grid_data = pd.concat(tmp_df_list, join="outer")

In [41]:
grid_data.head(10)

,filename,year,day,hour,grid_number,wave_height,wave_direction,wave_period
0,o202101612.0.wav,2021,16,13,1,0.150,128,1.6
1,o202101612.0.wav,2021,16,13,2,0.207,115,1.8
2,o202101612.0.wav,2021,16,13,3,0.288,96,2.2
3,o202101612.0.wav,2021,16,13,4,0.361,88,2.4
4,o202101612.0.wav,2021,16,13,5,0.168,77,1.6
5,o202101612.0.wav,2021,16,13,6,0.221,75,1.9
6,o202101612.0.wav,2021,16,13,7,0.266,75,2.1
7,o202101612.0.wav,2021,16,13,8,0.287,73,2.2
8,o202101612.0.wav,2021,16,13,9,0.303,76,2.2
9,o202101612.0.wav,2021,16,13,10,0.335,79,2.3


In [42]:
print("There are {} unique files in grid_data and {} rows".format(len(local_wav_files), len(grid_data)))

There are 32 unique files in grid_data and 143232 rows


# Q2c

In [43]:
ontario5km = open("ontario5km.map", "r").read().split("\n")

In [44]:
map_list = []
for item in ontario5km:
    data_l = item.split(" ")
    data_l = [i for i in data_l if i]
    
    grid_number = int(data_l[0])
    fortran_column = data_l[1]
    fortran_row = data_l[2]
    lat = float(data_l[3])
    lon = float(data_l[4])
    
    map_list.append((grid_number, fotran_column, fotran_row, lat, lon))
    
map_file = pd.DataFrame(map_list, columns=["grid_number", "fortran_column", "fortran_row", "lat", "lon"])

In [45]:
map_file['lon'] = map_file["lon"] * -1
map_file.head()

,grid_number,fortran_column,fortran_row,lat,lon
0,1,053,024,43.19300,-79.48237
1,2,053,024,43.19380,-79.42086
2,3,053,024,43.19456,-79.35935
3,4,053,024,43.19529,-79.29785
4,5,053,024,43.23541,-79.66814


# Q2d

In [46]:
grid_data.iloc[0, 4]

1

In [47]:
grid_data_final = pd.merge(grid_data, map_file[["grid_number", "lat", "lon"]], on = "grid_number")

In [50]:
grid_data_final.head()

,filename,year,day,hour,grid_number,wave_height,wave_direction,wave_period,lat,lon
0,o202101612.0.wav,2021,16,13,1,0.150,128,1.6,43.193,-79.48237
1,o202101612.0.wav,2021,16,13,1,0.140,129,1.5,43.193,-79.48237
2,o202101612.0.wav,2021,16,13,1,0.132,138,1.5,43.193,-79.48237
3,o202101612.0.wav,2021,16,13,1,0.147,128,1.5,43.193,-79.48237
4,o202101612.0.wav,2021,16,13,1,0.242,114,1.9,43.193,-79.48237


# Q2e

In [52]:
grid_data_final.to_csv("grid_data_final.csv", encoding="utf_8_sig", index=None)

# Q2f

In [59]:
# 宝贝 老师给的 utils.haversine() 这个方法应该是他自己定义的，你试试能不能用，不行的话，就用我写的这个 calculate_distance，都是计算距离的
import math

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0

    a = math.sin(dLat / 2) * math.sin(dLat / 2) + math.cos(lat1 * math.pi / 180.0) * math.cos(lat2 * math.pi / 180.0) * math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    dist = R * c
    return dist


In [75]:
facility_lat = 43.9593
facility_lon =  -78.1677
distance_list = []
for i in range(len(grid_data_final)):
    distance = calculate_distance(facility_lat, facility_lon, grid_data_final.loc[i, "lat"], grid_data_final.loc[i, "lon"])
    distance_list.append(distance)

In [77]:
grid_data_final["distance"] = distance_list

In [78]:
grid_data_final.head()

,filename,year,day,hour,grid_number,wave_height,wave_direction,wave_period,lat,lon,distance
0,o202101612.0.wav,2021,16,13,1,0.150,128,1.6,43.193,-79.48237,135.924536
1,o202101612.0.wav,2021,16,13,1,0.140,129,1.5,43.193,-79.48237,135.924536
2,o202101612.0.wav,2021,16,13,1,0.132,138,1.5,43.193,-79.48237,135.924536
3,o202101612.0.wav,2021,16,13,1,0.147,128,1.5,43.193,-79.48237,135.924536
4,o202101612.0.wav,2021,16,13,1,0.242,114,1.9,43.193,-79.48237,135.924536


# Q2g

In [86]:
sorted(grid_data_final["distance"].unique())[:3]

[4.575602589969131, 5.0480215339288685, 8.129956857628729]

In [134]:
closest_points = grid_data_final[grid_data_final["distance"] <= 8.129956857628729][["grid_number", "wave_height", "wave_direction", "wave_period", "lat", "lon"]]

In [135]:
closest_points.head()

,grid_number,wave_height,wave_direction,wave_period,lat,lon
126144,658,0.396,73,2.8,43.92229,-78.25527
126145,658,0.421,70,2.9,43.92229,-78.25527
126146,658,0.517,64,3.4,43.92229,-78.25527
126147,658,0.551,60,3.5,43.92229,-78.25527
126148,658,0.550,60,3.5,43.92229,-78.25527


In [100]:
print("closest_points has {} rows and {} unique grid number".format(closest_points.shape[0], len(closest_points['grid_number'].unique())))

closest_points has 576 rows and 3 unique grid number


In [136]:
# 宝贝 这部分实在是 眼睛不够用了 我都看花了 看样子应该是直接跑就行，你试试看
# Import 'grid_plot.csv'
grid_plot = pd.read_csv('grid_plot.csv')

# Create a map of Toronto
map2 = folium.Map(location=[43.9593, -78.1677], 
                 tiles='cartodbpositron', 
                 zoom_start=8)

# Add bike stations to the map
for idx, row in grid_plot.iterrows():
    folium.Circle(location=[row.lat, row.lon],
                  radius=20,
                  color='black').add_to(map2)

# Add bike stations to the map
for idx, row in closest_points.iterrows():
    folium.Circle(location=[row.lat, row.lon],
                  radius=20,
                  color='red').add_to(map2)
    
# Add weather stations
folium.Marker([43.6532, -79.3832], icon=folium.Icon(color='blue'), popup='UofT').add_to(map2)
folium.Marker([43.9593, -78.1677], icon=folium.Icon(color='red'), popup='Facility').add_to(map2)

# Display map
map2
                                   

# Q2h

In [112]:
import numpy as np

In [119]:
closest_points["wave_height"] = closest_points["wave_height"].astype(float)

In [123]:
wave_height_min = closest_points["wave_height"].min()
wave_height_max = closest_points["wave_height"].max()
wave_height_mean = closest_points["wave_height"].mean()

In [129]:
print("Wave height min: {} m\nWave height max {} m\nWave height mean {} m".format(wave_height_min, wave_height_max, wave_height_mean)) 

Wave height min: 0.063 m
Wave height max 2.629 m
Wave height mean 0.931701388888889 m
